In [ ]:
!pip install sentencepiece

In [ ]:
! pip install datasets transformers seqeval

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.30.0


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "Clinical-AI-Apollo/Medical-NER"
batch_size = 4

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
datasets = load_dataset("ktgiahieu/maccrobat2018_2020")
datasets = datasets['train'].train_test_split(test_size=0.1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 360
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 40
    })
})

In [ ]:
from datasets import Dataset, ClassLabel, Value, DatasetDict

# Define the ClassLabel feature with the label names
label_list = ['B-Activity', 'B-Administration', 'B-Age', 'B-Area', 'B-Biological_attribute', 'B-Biological_structure', 'B-Clinical_event', 'B-Color', 'B-Coreference', 'B-Date', 'B-Detailed_description', 'B-Diagnostic_procedure', 'B-Disease_disorder', 'B-Distance', 'B-Dosage', 'B-Duration', 'B-Family_history', 'B-Frequency', 'B-Height', 'B-History', 'B-Lab_value', 'B-Mass', 'B-Medication', 'B-Nonbiological_location', 'B-Occupation', 'B-Other_entity', 'B-Other_event', 'B-Outcome', 'B-Personal_background', 'B-Qualitative_concept', 'B-Quantitative_concept', 'B-Severity', 'B-Sex', 'B-Shape', 'B-Sign_symptom', 'B-Subject', 'B-Texture', 'B-Therapeutic_procedure', 'B-Time', 'B-Volume', 'B-Weight', 'I-Activity', 'I-Administration', 'I-Age', 'I-Area', 'I-Biological_attribute', 'I-Biological_structure', 'I-Clinical_event', 'I-Color', 'I-Coreference', 'I-Date', 'I-Detailed_description', 'I-Diagnostic_procedure', 'I-Disease_disorder', 'I-Distance', 'I-Dosage', 'I-Duration', 'I-Family_history', 'I-Frequency', 'I-Height', 'I-History', 'I-Lab_value', 'I-Mass', 'I-Medication', 'I-Nonbiological_location', 'I-Occupation', 'I-Other_entity', 'I-Other_event', 'I-Outcome', 'I-Personal_background', 'I-Qualitative_concept', 'I-Quantitative_concept', 'I-Severity', 'I-Shape', 'I-Sign_symptom', 'I-Subject', 'I-Texture', 'I-Therapeutic_procedure', 'I-Time', 'I-Volume', 'I-Weight', 'O']
label = ClassLabel(names=label_list)
# Define a new feature with the numeric labels
numeric_labels_feature = Value("int32")

In [ ]:
datasets = datasets.map(lambda example: {"tokens": example['tokens'],
    "tags": example["tags"],
    "numeric_tags": [label.encode_example(x) for x in example["tags"]],
})

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'numeric_tags'],
        num_rows: 360
    })
    test: Dataset({
        features: ['tokens', 'tags', 'numeric_tags'],
        num_rows: 40
    })
})

In [ ]:
# label_list = datasets["train"].features[f"{task}_tags"].feature.names
# label_list

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,tokens,tags,numeric_tags
0,"[A, 30, -, year, -, old, female, (, 65, kg, ), underwent, rhinoplasty, under, general, anaesthesia, ,, which, was, induced, using, a, combination, of, a, bolus, of, Remifentanyl, (, 0.5, μg, /, kg, ), and, Propofol, 2, mg, /, kg, ., \n, Muscle, paralysis, was, obtained, with, Cisatracurium, (, 0.15, μg, /, kg, ), ., \n, Transoral, intubation, was, performed, with, a, Macintosh, blade, (, No.3, ), and, a, size, 7, mm, diameter, reinforced, tracheal, tube, was, easily, and, gently, placed, in, the, trachea, ,, on, the, first, attempt, and, was, then, fixed, to, the, right, corner, of, the, mouth, ...]","[O, B-Age, I-Age, I-Age, I-Age, I-Age, B-Sex, O, O, O, O, O, B-Therapeutic_procedure, O, B-Medication, I-Medication, O, O, O, O, O, O, O, O, O, O, O, B-Medication, O, O, O, O, O, O, O, B-Medication, B-Dosage, I-Dosage, I-Dosage, I-Dosage, O, O, O, O, O, O, O, B-Medication, O, B-Dosage, I-Dosage, I-Dosage, I-Dosage, O, O, O, O, B-Therapeutic_procedure, O, O, O, O, B-Therapeutic_procedure, I-Therapeutic_procedure, O, B-Detailed_description, O, O, O, O, O, O, O, O, B-Therapeutic_procedure, I-Therapeutic_procedure, O, O, O, O, O, O, O, B-Biological_structure, O, O, O, O, O, O, O, O, O, O, O, B-Biological_structure, I-Biological_structure, I-Biological_structure, I-Biological_structure, I-Biological_structure, ...]","[81, 2, 43, 43, 43, 43, 32, 81, 81, 81, 81, 81, 37, 81, 22, 63, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 22, 81, 81, 81, 81, 81, 81, 81, 22, 14, 55, 55, 55, 81, 81, 81, 81, 81, 81, 81, 22, 81, 14, 55, 55, 55, 81, 81, 81, 81, 37, 81, 81, 81, 81, 37, 77, 81, 10, 81, 81, 81, 81, 81, 81, 81, 81, 37, 77, 81, 81, 81, 81, 81, 81, 81, 5, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 5, 46, 46, 46, 46, ...]"
1,"[A, 74, -, year, -, old, man, was, referred, to, our, hospital, in, November, 2000, because, of, liver, dysfunction, detected, during, a, medical, checkup, ., \n, The, patient, had, been, diagnosed, with, nephrotic, syndrome, in, 1995, ., \n, Laboratory, examinations, showed, elevated, serum, hepatobiliary, enzymes, and, IgM, ,, and, the, presence, of, antimitochondrial, antibodies, ., \n, Serologic, markers, for, Hepatitis, B, and, C, viruses, were, negative, ., \n, Histopathologic, examination, of, a, liver, biopsy, specimen, obtained, at, laparoscopy, revealed, non, -, suppurative, destructive, cholangitis, in, the, portal, area, (, Figure, ​1, ), ., \n, The, diagnosis, of, PBC, (, Scheuer, ...]","[O, B-Age, I-Age, I-Age, I-Age, I-Age, B-Sex, O, B-Clinical_event, O, O, B-Nonbiological_location, O, B-Date, I-Date, O, O, B-Sign_symptom, I-Sign_symptom, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Disease_disorder, I-Disease_disorder, O, B-Date, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, I-Diagnostic_procedure, O, B-Diagnostic_procedure, O, O, O, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, O, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, B-Diagnostic_procedure, O, O, B-Lab_value, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, O, B-Diagnostic_procedure, I-Diagnostic_procedure, O, O, B-Therapeutic_procedure, O, O, O, O, O, B-Disease_disorder, O, O, B-Biological_structure, I-Biological_structure, O, O, O, O, O, O, O, O, O, B-Disease_disorder, O, B-Disease_disorder, ...]","[81, 2, 43, 43, 43, 43, 32, 81, 6, 81, 81, 23, 81, 9, 50, 81, 81, 34, 74, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 12, 53, 81, 9, 81, 81, 11, 52, 81, 81, 11, 52, 52, 81, 11, 81, 81, 81, 81, 81, 11, 52, 81, 81, 81, 81, 81, 11, 52, 81, 11, 81, 81, 20, 81, 81, 11, 52, 81, 81, 81, 11, 52, 81, 81, 37, 81, 81, 81, 81, 81, 12, 81, 81, 5, 46, 81, 81, 81, 81, 81, 81, 81, 81, 81, 12, 81, 12, ...]"
2,"[The, 31, -, year, -, old, white, primipara, with, twin, pregnancy, was, admitted, to, hospital, in, the, 38th, week, of, gestation, with, elevated, blood, pressure, (, 150/100, mmHg, ), ., \n, After, receiving, antihypertensive, tre

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [1, 5365, 261, 291, 269, 311, 4378, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [1, 5365, 366, 291, 269, 311, 4378, 3671, 352, 1023, 323, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['The', '36', '-', 'year', '-', 'old', 'male', 'patient', 'in', 'this', 'case', 'had', 'a', '6', '-', 'year', 'history', 'of', 'diffuse', 'cutaneous', 'SSc', '(', 'Fig.1', ')', 'and', 'had', 'not', 'received', 'regular', 'follow', '-', 'up', 'or', 'medication', 'therapy', '.', '\n', 'He', 'had', 'experienced', 'malaise', ',', 'poor', 'appetite', ',', 'and', 'progressive', 'shortness', 'of', 'breath', 'for', '1', 'month', 'and', 'oligouria', 'for', '3', 'days', '.', '\n', 'He', 'was', 'admitted', 'to', 'our', 'hospital', 'with', 'renal', 'failure', 'and', 'pulmonary', 'edema', '.', '\n', 'In', 'the', 'emergency', 'department', ',', 'his', 'blood', 'pressure', ',', 'pulse', 'rate', ',', 'and', 'respiratory', 'rate', 'were', '174/127', 'mm', 'Hg', ',', '88', '/', 'min', ',', 'and', '22', '/', 'min', ',', 'respectively', '.', '\n', 'Laboratory', 'analysis', 'revealed', 'the', 'following', 'values', ':', 'white', 'blood', 'cell', 'count', ',', '7730', '/', 'μL', '(', '3500–11,000', '/', 'μL

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '▁The', '▁36', '▁-', '▁year', '▁-', '▁old', '▁male', '▁patient', '▁in', '▁this', '▁case', '▁had', '▁a', '▁6', '▁-', '▁year', '▁history', '▁of', '▁diffuse', '▁cutaneous', '▁SS', 'c', '▁(', '▁Fig', '.', '1', '▁)', '▁and', '▁had', '▁not', '▁received', '▁regular', '▁follow', '▁-', '▁up', '▁or', '▁medication', '▁therapy', '▁.', '▁He', '▁had', '▁experienced', '▁malaise', '▁,', '▁poor', '▁appetite', '▁,', '▁and', '▁progressive', '▁shortness', '▁of', '▁breath', '▁for', '▁1', '▁month', '▁and', '▁oligo', 'uria', '▁for', '▁3', '▁days', '▁.', '▁He', '▁was', '▁admitted', '▁to', '▁our', '▁hospital', '▁with', '▁renal', '▁failure', '▁and', '▁pulmonary', '▁edema', '▁.', '▁In', '▁the', '▁emergency', '▁department', '▁,', '▁his', '▁blood', '▁pressure', '▁,', '▁pulse', '▁rate', '▁,', '▁and', '▁respiratory', '▁rate', '▁were', '▁174', '/', '127', '▁mm', '▁Hg', '▁,', '▁88', '▁/', '▁min', '▁,', '▁and', '▁22', '▁/', '▁min', '▁,', '▁respectively', '▁.', '▁Laboratory', '▁analysis', '▁revealed', '▁the', 

In [ ]:
len(example[f"numeric_tags"]), len(tokenized_input["input_ids"])

(816, 901)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 20, 21, 22, 22, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 90, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 118, 119, 120, 120, 121, 122, 122, 122, 122, 122, 123, 124, 124, 125, 126, 127, 128, 129, 129, 129, 131, 132, 133, 133, 134, 135, 135, 135, 137, 138, 139, 139, 140, 140, 141, 142, 143, 144, 144, 145, 146, 146, 147, 148, 148, 148, 148, 148, 148, 148, 149, 150, 150, 151, 152, 153, 154, 155, 156, 157, 159, 160, 161, 161, 162, 163, 163, 163, 165, 166, 167, 167, 168, 169, 170, 171, 172, 172, 172, 174, 175, 176, 176, 177, 178, 178, 178, 178, 178, 178, 178, 180, 181, 182, 182, 183, 184, 185, 186, 187, 1

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"numeric_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

901 901


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"numeric_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[1, 336, 969, 341, 395, 341, 597, 102428, 2844, 1992, 275, 266, 2785, 5142, 265, 24998, 2345, 267, 462, 1365, 323, 383, 284, 6524, 275, 266, 119935, 60930, 267, 737, 4406, 323, 383, 284, 1338, 288, 1384, 275, 1697, 2665, 1272, 323, 443, 284, 363, 820, 265, 33110, 42093, 289, 13488, 1682, 323, 329, 1799, 330, 8969, 830, 265, 266, 119935, 60930, 287, 584, 260, 473, 260, 366, 782, 24772, 366, 29418, 14922, 18886, 366, 782, 26912, 455, 6937, 366, 614, 1312, 366, 27268, 4861, 510, 366, 263, 2455, 19038, 3579, 1263, 647, 5237, 376, 6311, 323, 855, 3499, 284, 12335, 5036, 323, 1652, 12095, 3019, 50347, 284, 602, 320, 4285, 267, 262, 423, 1572, 275, 76657, 260, 3935, 1566, 376, 260, 1883, 10246, 1627, 8734, 263, 602, 320, 4285, 267, 262, 595, 1572, 275, 4853, 877, 1566, 693, 260, 1883, 1566, 767, 260, 1794, 10246, 8535, 8734, 323, 50255, 332, 3100, 267, 884, 366, 1489, 263, 18312, 264, 731, 275, 363, 5315, 95922, 18336, 57077, 13543, 323, 106828, 58738, 284, 1697, 267, 462, 1365

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list) + 1)

# old_model= AutoModelForTokenClassification.from_pretrained(model_checkpoint)
# model= AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_list))
# model.bert=old_model.bert

In [ ]:
datasets['train'].shape

(360, 3)

In [ ]:
!pip install transformers==4.30

In [ ]:
!pip install accelerate -U

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

<ipython-input-38-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
labels = [label_list[i] for i in example[f"numeric_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'Age': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Biological_structure': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 3},
 'Clinical_event': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 'Coreference': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Date': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 6},
 'Detailed_description': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 5},
 'Diagnostic_procedure': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 36},
 'Disease_disorder': {'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0,
  'number': 11},
 'Dosage': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Duration': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'Frequency': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Lab_value': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 22},
 'Medication': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'numb

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/trung11/Medical-NER-finetuned-ner into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.917413,0.423864,0.361317,0.390099,0.744790
2,No log,0.681428,0.525669,0.552147,0.538583,0.789906
3,No log,0.626160,0.538278,0.726510,0.618387,0.797398
4,No log,0.493396,0.606500,0.729093,0.662170,0.843359
5,No log,0.507073,0.610216,0.794640,0.690323,0.843078
6,0.784700,0.419520,0.686335,0.796254,0.737220,0.874395
7,0.784700,0.421470,0.685044,0.838553,0.754065,0.881604
8,0.784700,0.380664,0.728743,0.844043,0.782166,0.898502
9,0.784700,0.347357,0.738263,0.847917,0.789300,0.907908
10,0.784700,0.325903,0.758251,0.867937,0.809395,0.913484


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=1800, training_loss=0.3345100996229384, metrics={'train_runtime': 1378.1152, 'train_samples_per_second': 5.225, 'train_steps_per_second': 1.306, 'total_flos': 1879338734062872.0, 'train_loss': 0.3345100996229384, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.31137973070144653,
 'eval_precision': 0.790308869368093,
 'eval_recall': 0.9005489183080401,
 'eval_f1': 0.8418351946875944,
 'eval_accuracy': 0.931339416469528,
 'eval_runtime': 2.5175,
 'eval_samples_per_second': 15.889,
 'eval_steps_per_second': 3.972,
 'epoch': 20.0}